<a href="https://colab.research.google.com/github/Navenkumar-Balasubramaniam/00-General/blob/main/01%20Python%20API%20Project/01_bookapi_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install flask cloudflared

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 2.6 MB/s eta 0:00:00


The ! lets us run shell commands in Google Colab.

pip install adds two Python packages:

Flask → the micro web framework we’ll use to build a REST API.

cloudflared → lets us create a free, temporary public URL (a “tunnel”) so the API notebook can be reached from the second notebook.

-q just makes the install quieter (less output).

In [2]:
import sqlite3, os

Imports the built-in SQLite library (no need to install anything) to manage a small file-based database.

Imports os to handle file paths cleanly.

In [3]:
con = sqlite3.connect('db.sqlite'); cur = con.cursor()

Opens (or creates) a file named db.sqlite.

Returns a connection object (con) and a cursor (cur) used to send SQL commands to the database.

In [4]:
cur.execute("CREATE TABLE IF NOT EXISTS books(id INTEGER PRIMARY KEY, title TEXT, author TEXT, year INTEGER)")


Runs an SQL command to create a table named books if it doesn’t already exist.

The table columns are:

id (integer primary key — unique ID for each book)

title (text)

author (text)

year (integer)

In [6]:
seed = [(1,'1984','George Orwell',1949),(2,'Dune','Frank Herbert',1965)]

A simple list of tuples — our sample data (two classic books).

In [7]:
cur.executemany("INSERT OR IGNORE INTO books VALUES(?,?,?,?)", seed)

Adds both records into the table.

INSERT OR IGNORE ensures we don’t insert duplicates if this cell runs more than once.

? placeholders protect against SQL injection and handle data safely.

In [8]:
con.commit(); con.close()
print("SQLite ready at", os.path.abspath("db.sqlite"))


SQLite ready at /content/db.sqlite


Saves all changes to disk.

Closes the connection cleanly (good practice).

Confirms that our SQLite database file was created and shows its absolute path (useful for debugging or referencing later).

In [10]:
from flask import Flask, jsonify, request

In [11]:
import threading

Imports

Flask → lightweight web server.

jsonify → turns Python objects into JSON.

request → lets us handle data sent by clients.

sqlite3, threading, os → database + background thread.

A thread is like a mini-program running inside your main Python program —
it lets multiple parts of your code run at the same time (concurrently).

🔍 Why we need it here

When you call:

app.run()


Flask starts a web server that never stops — it keeps listening for incoming requests.
If you run this normally in a Colab cell, the cell would be stuck forever showing the server logs,
and you couldn’t run any other cells.

So we do this instead:

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=8000)).start()


That means:

threading.Thread(...) creates a new thread that runs the Flask server.

.start() launches it in the background.

The notebook cell finishes immediately, so you can still run new cells while the server stays active.

In [12]:
app = Flask(__name__)

def get_db():
    con = sqlite3.connect('db.sqlite')
    con.row_factory = sqlite3.Row   # rows behave like dicts
    return con

@app.route('/books', methods=['GET'])
def get_books():
    con = get_db()
    books = con.execute('SELECT * FROM books').fetchall()
    con.close()
    return jsonify([dict(b) for b in books])

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=8000)).start()

Creates a Flask app (app = Flask(__name__)).

Defines a helper get_db() to open the db.sqlite file each time we need it.

row_factory makes query results behave like dictionaries (so we can easily convert to JSON).

Defines one route so far:

GET /books returns all rows from the books table as JSON.

Runs Flask in a background thread so the Colab cell doesn’t block.

In [14]:
# 1) install cloudflared binary
!wget -q -O /tmp/cloudflared.deb "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
!apt install -y /tmp/cloudflared.deb


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cloudflared' instead of '/tmp/cloudflared.deb'
The following NEW packages will be installed:
  cloudflared
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 0 B/20.2 MB of archives.
After this operation, 41.3 MB of additional disk space will be used.
Get:1 /tmp/cloudflared.deb cloudflared amd64 2025.11.1 [20.2 MB]
Selecting previously unselected package cloudflared.
(Reading database ... 121229 files and directories currently installed.)
Preparing to unpack /tmp/cloudflared.deb ...
Unpacking cloudflared (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [15]:
# 2) check it's available and show version
!which cloudflared && cloudflared --version


/usr/local/bin/cloudflared
cloudflared version 2025.11.1 (built 2025-11-07-16:59 UTC)


In [16]:
# 3) re-run the tunnel starter (same logic as before)
from pathlib import Path; LOG = Path('/content/tunnel.log')
import subprocess, time, re
subprocess.Popen(['cloudflared','tunnel','--url','http://localhost:8000','--no-autoupdate'],
                 stdout=open(LOG,'w'), stderr=subprocess.STDOUT)
time.sleep(4)
m = re.search(r'https://[-a-z0-9]+\.trycloudflare\.com', LOG.read_text())
print('Public URL:', m.group(0) if m else 'Still starting… re-run this cell.')


Public URL: Still starting… re-run this cell.
